I think that an efficient way to find the tags of #book inside my 
whole obsidian vault is by using grep, but as this is a python 
programme, I don't know if I can insert shell execution in here. 
I should probably be able to do so if there is a built library.

In [1]:
import subprocess
books = ""
# books.join(subprocess.run(['shell', 'rg -r "#book" home/schronding/Documents/clean_sync']))



There is indeed a library called subprocess that is  able to 
access the terminal. It seems that with subprocess.run() I can
run any command from the shell. I think that the command should
look something like 
'rg -r "#book" home/schronding/Documents/clean_sync'


Here my purpose is to insert the list of the books as a string 
inside the variable 'books'. 
FileNotFoundError: [Errno 2] No such file or directory: 'shell'
It seems like there is no 'shell'. It probably goes by another name. 
It seems that what is missing is the argument 'shell=True' in the
method '.run()'

In [ ]:
subprocess.run(['shell', 'rg -r "#book" home/schronding/Documents/clean_sync'], shell=True)

The command seems to work, but I don't know if I am missing the syntax
of ripgrep, or if I wrote the routed incorrectly. 
pwd returned /home/schronding/Documents/clean_sync

In [ ]:
subprocess.run(['shell', 'rg -r "#book" /home/schronding/Documents/clean_sync'], shell=True)


The error 
'rg -r "#book" /home/schronding/Documents/clean_sync: 1: shell: not found' 
persists. I might be messing up with the ripgrep syntax
It seems thar ripgrep uses a syntax similar to sed. The thing I have 
notice is that even if the command works, it will probably just return the 
line that contains the hashtag '#book', but what I want is the title
of the note. I will assume that the titles are on the first line of the
markdown file. I don't how I can narrow the search of ripgrep, but at 
least I found a command that I think will work

In [ ]:

subprocess.run(['shell', 'rg --files | rg #book'], shell=True)

It stil says that the shell hasn't been found
rg --files | rg #book: 1: shell: not found
It seems I indeed need to wrap the searching term between quotes 
so ripgrep can found it. While it seems that the command worked, it 
didn't print anything, which is weird 
rg --files | rg #book
rg: ripgrep requires at least one pattern to execute a search
rg --files | rg "#book"
Nothing
It seems I was overcomplicating things as simply going to the folder where
I want to use rg is enough for the program to look recursively on all the 
children folders. It also provides me the with the title of the note, so I don't
need to do anything special, the correct line of code is 

In [2]:
subprocess.run(['rg "#book" /home/schronding/Documents/clean_sync'], shell = True)

/home/schronding/Documents/clean_sync/2_Areas/books/how_to_take_smart_notes/How to take smart notes.md:#book 
/home/schronding/Documents/clean_sync/2_Areas/books/the_simple_path_to_wealth/the simple path to wealth.md:#book 
/home/schronding/Documents/clean_sync/2_Areas/books/essentialism/Essentialism.md:#book 
/home/schronding/Documents/clean_sync/2_Areas/books/dont_die/Don't die.md:#book 
/home/schronding/Documents/clean_sync/2_Areas/books/endure_cameron_hanes/Endure (Cameron Hanes).md:#book 
/home/schronding/Documents/clean_sync/2_Areas/books/doing_good_better/Doing Good Better.md:#book 
/home/schronding/Documents/clean_sync/2_Areas/Interpersonal behavior/You can have wealthy girlfriends.md:#book 
/home/schronding/Documents/clean_sync/2_Areas/Interpersonal behavior/We should reward people when we like something they do.md:#book 
/home/schronding/Documents/clean_sync/2_Areas/books/ultralearning/Ultralearning.md:#book 
/home/schronding/Documents/clean_sync/2_Areas/books/meditations_for

CompletedProcess(args=['rg "#book" /home/schronding/Documents/clean_sync'], returncode=0)

It worked! Now I can see the list of notes that have the tag #book in them. The problem is that it also prints the direction and also the tag at the end. 
I think I might be able to use sed to have just the title of the notes. My idea is to use regex to return the title of the note. Taking what comes after the last 
'/' sounds like a good idea. Erasing what comes after the dot (.) will allow me to remove the tags, as there are some examples in which other tag is put in the same
line as #book

In [5]:
list_of_books = subprocess.run('rg -l "#book" /home/schronding/Documents/clean_sync', shell=True, capture_output=True, text=True)
list_of_books.stdout

"/home/schronding/Documents/clean_sync/2_Areas/books/endure_cameron_hanes/Endure (Cameron Hanes).md\n/home/schronding/Documents/clean_sync/2_Areas/books/ultralearning/Ultralearning.md\n/home/schronding/Documents/clean_sync/2_Areas/books/educated/Educated.md\n/home/schronding/Documents/clean_sync/2_Areas/books/psychedelic_explorer_journey/the psychedelic explorer's guide.md\n/home/schronding/Documents/clean_sync/2_Areas/books/how_to_take_smart_notes/How to take smart notes.md\n/home/schronding/Documents/clean_sync/2_Areas/books/the_simple_path_to_wealth/the simple path to wealth.md\n/home/schronding/Documents/clean_sync/2_Areas/books/essentialism/Essentialism.md\n/home/schronding/Documents/clean_sync/2_Areas/Interpersonal behavior/You can have wealthy girlfriends.md\n/home/schronding/Documents/clean_sync/2_Areas/Interpersonal behavior/We should reward people when we like something they do.md\n/home/schronding/Documents/clean_sync/2_Areas/Interpersonal behavior/You can encourage behavior

The magic of using the -l flag is that it gives me the path/name of the file
that contains #book, not the whole line. In this way I save myself the effort of using sed to remove it afterwards. It is the same that '--files-with-matches'
'capture_output=True' is what I needed so python receives the information, as normally the method of .run() just prints to the terminal the result. 
Finally the argument 'text=True' ensures I get a string, not raw bytes. 
What I need to do next is to separate the string into its constituent parts, which seems that can be done with the method '.splitlines()'

In [10]:
list_of_books.splitlines

AttributeError: 'CompletedProcess' object has no attribute 'splitlines'

It seems that splitlines is not a method of strings, which is curious because the error says that it is an object

AttributeError: 'CompletedProcess' object has no attribute 'splitlines'

I suppose that while I asked for text in the argument there might be some extra information or methods that transform 'list_of_books' into an object. It makes sense as the method .stdout() doesn't exist in strings... does it? 

In [9]:
test_string = "This is just to try the .stdout() method"
test_string.stdout

AttributeError: 'str' object has no attribute 'stdout'

Indeed it hasn't 

AttributeError: 'str' object has no attribute 'stdout'

Yet it seems that splitlines() IS a method for strings


In [11]:
test_string.splitlines()

['This is just to try the .stdout() method']

It indeed does! Then if I can my object list_of_books to string I might be done

In [13]:
string_of_books = str(list_of_books)
list_of_books = string_of_books.splitlines()
print(list_of_books)

['[\'CompletedProcess(args=\\\'rg -l "#book" /home/schronding/Documents/clean_sync\\\', returncode=0, stdout="/home/schronding/Documents/clean_sync/2_Areas/books/endure_cameron_hanes/Endure (Cameron Hanes).md\\\\n/home/schronding/Documents/clean_sync/2_Areas/books/ultralearning/Ultralearning.md\\\\n/home/schronding/Documents/clean_sync/2_Areas/books/educated/Educated.md\\\\n/home/schronding/Documents/clean_sync/2_Areas/books/psychedelic_explorer_journey/the psychedelic explorer\\\'s guide.md\\\\n/home/schronding/Documents/clean_sync/2_Areas/books/how_to_take_smart_notes/How to take smart notes.md\\\\n/home/schronding/Documents/clean_sync/2_Areas/books/the_simple_path_to_wealth/the simple path to wealth.md\\\\n/home/schronding/Documents/clean_sync/2_Areas/books/essentialism/Essentialism.md\\\\n/home/schronding/Documents/clean_sync/2_Areas/Interpersonal behavior/You can have wealthy girlfriends.md\\\\n/home/schronding/Documents/clean_sync/2_Areas/Interpersonal behavior/We should reward p

While it worked I don't know if it is what I want, as I see I lot of backslashs, which makes me think there was a lot of escaping, and while I understand it with characters like (') I don't know if I have that many characteres that need to be escaped. I will continue with sed though

In [14]:
subprocess.run("sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books")

FileNotFoundError: [Errno 2] No such file or directory: "sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books"

There are a lot of errors, but I think my syntax for sed is correct, the thing is that I am trying to reference a local variable instead of a file. I will save it with json too see how it works

In [15]:
import json

with open("list_of_books.json", "w") as file:
    json.dump(list_of_books, file, indent=4)

It worked! Now the thing is that it is inside square brackets, and I don't know if that means a literal python list

In [16]:
subprocess.run("sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books")

FileNotFoundError: [Errno 2] No such file or directory: "sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books"

It still has an error 

FileNotFoundError: [Errno 2] No such file or directory: "sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books"

I think that I need to escape the dashes, as that might be messing up with sed

In [19]:
subprocess.run("sed -i 's/\/home\/schronding\/Documents\/clean_sync\///g' list_of_books")

FileNotFoundError: [Errno 2] No such file or directory: "sed -i 's/\\/home\\/schronding\\/Documents\\/clean_sync\\///g' list_of_books"

It seems that escaping the dashes didn't worked. I think that maybe I could store the path to a variable in order to not worry about messing up with the file and the brackets for substitution

In [20]:
clean_path = "/home/schronding/Documents/clean_sync/"

Now let's see if I can use f strings with subprocess.run()

In [21]:
subprocess.run(f"sed -i 's/{clean_path}//g' list_of_books.json")

FileNotFoundError: [Errno 2] No such file or directory: "sed -i 's//home/schronding/Documents/clean_sync///g' list_of_books.json"